In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import pyodbc

In [3]:
# !python -m pip install geopandas

In [4]:
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=prdsql05.westfund.com.au;'
    'DATABASE=BRONZE;'
    'Trusted_Connection=yes;'
)

In [5]:
input_folder = r"\\prdeqs01\QlikData\GEO\input"
output_folder = r"\\prdeqs01\QlikData\GEO\output"
local_ = r"C:\Users\zengsh\OneDrive - Westfund Ltd\Documents\Westfund_"

In [6]:
query = """
SELECT *
  FROM [GOLD].[dbo].[Membership_Reporting]
  WHERE [Run Month Num] = (SELECT MAX([Run Month Num]) FROM [GOLD].[dbo].[Membership_Reporting])
    AND [Product Cover] <> 'Overseas' AND [active_count] <> 0
"""

df_mem = pd.read_sql(query, conn)
len(df_mem)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_10116\720937309.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mem = pd.read_sql(query, conn)


67915

In [7]:
df_mem.head()

,Run Month,Run Month Num,Fin Year,Fin Month,Fin Month Order,MembershipKey,Membership Number,Membership Status,Product Cover,Membership Cover,...,Previous Fund,Join Reason,Termination Code,Termination Reason,Transfered Fund Id,Transfered Fund,active_count,join_count,suspended_count,terminated_count
0,2025-12-31,61,2026,Dec,6,52360PK 2025-12-31,52360.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
1,2025-12-31,61,2026,Dec,6,52368PK 2025-12-31,52368.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
2,2025-12-31,61,2026,Dec,6,52555PK 2025-12-31,52555.0,A,Combined,Hosp/Extras,...,Other,Transfer from another fund,None,None,NaN,None,1,0,0,0
3,2025-12-31,61,2026,Dec,6,52537PK 2025-12-31,52537.0,A,Combined,Hosp/Extras,...,nib,Transfer from another fund,None,None,NaN,None,1,0,0,0
4,2025-12-31,61,2026,Dec,6,52542PK 2025-12-31,52542.0,A,Combined,Hosp/Extras,...,nib,Transfer from another fund,None,None,NaN,None,1,0,0,0


In [8]:
# 检查 Membership Number 列的重复项
print("总行数:", len(df_mem))
print("唯一的 Membership Number 个数:", df_mem['Membership Number'].nunique())
print("重复项个数:", df_mem['Membership Number'].duplicated().sum())

# 如果有重复项，显示详细信息
if df_mem['Membership Number'].duplicated().sum() > 0:
    print("\n重复的 Membership Number:")
    duplicates = df_mem[df_mem['Membership Number'].duplicated(keep=False)].sort_values('Membership Number')
    print(duplicates[['Membership Number']].drop_duplicates()['Membership Number'].value_counts())
else:
    print("\n✓ 没有重复的 Membership Number")

总行数: 67915
唯一的 Membership Number 个数: 67915
重复项个数: 0

✓ 没有重复的 Membership Number


In [9]:
query = """
SELECT *
  FROM [BRONZE].[dbo].[PersonAddressHomePostal]
"""

df = pd.read_sql(query, conn)
len(df)

C:\Users\zengsh\AppData\Local\Temp\ipykernel_10116\2897101254.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


172317

In [10]:
# 使用 df_mem 的 Membership Number 过滤已加载的 PersonAddressHomePostal 表（保留原始 `df`）
# 确保两个列的数据类型一致（转换为字符串）
mems = df_mem['Membership Number'].dropna().astype(str).unique()
print(f"使用 {len(mems)} 个唯一 Membership Number 进行过滤。原始地址表行数: {len(df)}")

# 将 membership_id 转为字符串以便比较
if 'membership_id' in df.columns:
    df['membership_id'] = df['membership_id'].astype(str)
else:
    raise KeyError('列 membership_id 在地址表中未找到')

# 过滤
df_address_filtered = df[df['membership_id'].isin(mems)].copy()
print(f"过滤后行数: {len(df_address_filtered)}")
print(f"过滤后唯一的 membership_id 个数: {df_address_filtered['membership_id'].nunique()}")



使用 67915 个唯一 Membership Number 进行过滤。原始地址表行数: 172317
过滤后行数: 70383
过滤后唯一的 membership_id 个数: 67915


In [11]:
df_address_filtered

,person_id,membership_id,relationship,state,address_type,address_line1,address_line2,address_line3,suburb,postcode,...,pstateid,paddress_type,paddress_line1,paddress_line2,paddress_line3,psuburb,ppostcode,plocality,addressid,paddressid
1,49.0,57.0,1,NSW,H,4 Brushtail Place,None,None,FULLERTON COVE,2318.0,...,None,None,None,None,None,None,NaN,None,266576.0,NaN
4,142.0,154.0,1,NSW,H,3 Suvla Street,None,None,LITHGOW,2790.0,...,None,None,None,None,None,None,NaN,None,59.0,NaN
8,228.0,276.0,1,NSW,H,46 Wallerawang Road,None,None,PORTLAND,2847.0,...,None,None,None,None,None,None,NaN,None,92.0,NaN
9,235.0,282.0,1,NSW,H,Ben-Ay Quarry Road,None,None,BEN BULLEN,2790.0,...,None,None,None,None,None,None,NaN,None,96.0,NaN
13,309.0,386.0,1,NSW,H,45 Andrew Street,None,None,LITHGOW,2790.0,...,None,None,None,None,None,None,NaN,None,114926.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172312,349642.0,200914.0,1,VIC,H,43 Harold Street,,,WENDOUREE,3355.0,...,None,None,None,None,None,None,NaN,None,452413.0,NaN
172313,349648.0,200917.0,1,QLD,H,106 Panorama Dr,,,NAMBOUR,4560.0,...,None,None,None,None,None,None,NaN,None,347694.0,NaN
172314,349688.0,200930.0,1,QLD,H,10 Montcliff Court,,,SPRINGFIELD,4300.0,...,QLD,P,10 Montcliff Court,,,SPRINGFIELD,4300.0,SPRINGFIELD,347748.0,347749.0
172315,349707.0,200937.0,1,NSW,H,84 Wallace Street South,,,COOLAMON,2701.0,...,None,None,None,None,None,None,NaN,None,451937.0,NaN


In [12]:
# 显示 GNAF_CORE 的前10行
# gnaf_file = os.path.join(input_folder, "GNAF_CORE.psv")
gnaf_file = os.path.join(local_, "GNAF_CORE.psv")
print("GNAF_CORE.psv 前10行:\n")
df_gnaf_head = pd.read_csv(gnaf_file, sep='|', nrows=100)
df_gnaf_head

GNAF_CORE.psv 前10行:



,ADDRESS_DETAIL_PID,DATE_CREATED,ADDRESS_LABEL,ADDRESS_SITE_NAME,BUILDING_NAME,FLAT_TYPE,FLAT_NUMBER,LEVEL_TYPE,LEVEL_NUMBER,NUMBER_FIRST,...,POSTCODE,LEGAL_PARCEL_ID,MB_CODE,ALIAS_PRINCIPAL,PRINCIPAL_PID,PRIMARY_SECONDARY,PRIMARY_PID,GEOCODE_TYPE,LONGITUDE,LATITUDE
0,GATAS702553725,15-02-2005,"UNIT 2 50 PRINCES ST, SANDY BAY TAS 7005",NaN,NaN,UNIT,2,NaN,NaN,50,...,7005,37014/1,60059230000,PRINCIPAL,NaN,SECONDARY,GATAS702332412,PROPERTY CENTROID,147.322658,-42.897346
1,GATAS702448146,13-10-2017,"UNIT 22 113 CHAPEL ST, GLENORCHY TAS 7010",NaN,NaN,UNIT,22,NaN,NaN,113,...,7010,237114/2,60054370000,PRINCIPAL,NaN,SECONDARY,GATAS702211395,PROPERTY CENTROID,147.265476,-42.840058
2,GANT_718977773,19-01-2018,"UNIT 38 117 MITCHELL ST, LARRAKEYAH NT 0820",NaN,NaN,UNIT,38,NaN,NaN,117,...,820,NaN,70008520000,PRINCIPAL,NaN,SECONDARY,GANT_703846938,STREET LOCALITY,130.832559,-12.456043
3,GATAS702765642,23-04-2018,"UNIT 3 2-6 PINKARD ST, KINGS MEADOWS TAS 7249",NaN,NaN,UNIT,3,NaN,NaN,2,...,7249,42688/3,60089930000,PRINCIPAL,NaN,SECONDARY,GATAS702318480,BUILDING CENTROID,147.164258,-41.472291
4,GATAS717990791,08-04-2011,"PIERMONT RESORT UNIT 9 12990 TASMAN HWY, SWANS...",NaN,PIERMONT RESORT,UNIT,9,NaN,NaN,12990,...,7190,139010/9,60049060000,PRINCIPAL,NaN,SECONDARY,GATAS717840912,PROPERTY CENTROID,148.076291,-42.155799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,GANSW715373670,11-07-2009,"UNIT 3 35 GRAHAM AV, CASULA NSW 2170",NaN,NaN,UNIT,3,NaN,NaN,35,...,2170,3/SP70981,10482850000,PRINCIPAL,NaN,SECONDARY,GANSW704376096,PROPERTY CENTROID,150.909291,-33.939950
96,GANSW717788499,27-07-2021,"HORDERN PLACE CARSPACE 262 73 VICTORIA ST, POT...",HORDERN PLACE,NaN,CARSPACE,262,NaN,NaN,73,...,2011,262/SP22353,11208790000,PRINCIPAL,NaN,SECONDARY,GANSW705743754,PROPERTY CENTROID,151.222672,-33.870232
97,GANSW705961509,11-07-2009,"UNIT 1 69 TOPHAM RD, SMEATON GRANGE NSW 2567",NaN,NaN,UNIT,1,NaN,NaN,69,...,2567,1/SP61086,10153383000,PRINCIPAL,NaN,SECONDARY,GANSW705951246,PROPERTY CENTROID,150.756960,-34.036574
98,GATAS719497140,16-10-2017,"SNODGRASS PARK UNIT 14 31 EDWARD ST, CURRIE TA...",NaN,SNODGRASS PARK,UNIT,14,NaN,NaN,31,...,7256,156961/1,60069550000,PRINCIPAL,NaN,SECONDARY,GATAS702777764,PROPERTY CENTROID,143.855769,-39.931040


In [ ]:
# 1️⃣ 参数配置
# =============================
df_match_cols = ['state', 'suburb', 'postcode']
gnaf_match_cols = ['STATE', 'LOCALITY_NAME', 'POSTCODE']

# gnaf_file = os.path.join(input_folder, "GNAF_CORE.psv")
gnaf_file = os.path.join(local_, "GNAF_CORE.psv")
chunksize = 500_000

# =============================
# 2️⃣ 整合 df_address_filtered 地址
# =============================
df_address_filtered['address_full'] = (
    df_address_filtered[['address_line1', 'address_line2', 'address_line3']]
    .fillna('')
    .astype(str)
    .agg(', '.join, axis=1)
    .str.replace(r'(,\s*)+', ', ', regex=True)
    .str.strip(', ')
)

# 统一为大写，后面不用反复 upper
df_address_filtered['address_full_u'] = df_address_filtered['address_full'].str.upper()

# =============================
# 3️⃣ postcode 统一为字符串（修正浮点数问题）
# =============================
if 'postcode' in df_address_filtered.columns:
    df_address_filtered['postcode'] = (
        df_address_filtered['postcode']
        .fillna(0)       # 空值填 0
        .astype(int)     # 去掉小数点
        .astype(str)     # 转字符串
    )

# =============================
# 4️⃣ 初始化经纬度
# =============================
df_address_filtered['LONGITUDE'] = np.nan
df_address_filtered['LATITUDE'] = np.nan

# =============================
# 5️⃣ 分块读取 GNAF 并做向量化匹配（仅街道名匹配）
# =============================
for chunk in pd.read_csv(gnaf_file, sep='|', chunksize=chunksize, dtype=str):

    # ---------- 基础清洗 ----------
    for col in ['POSTCODE', 'STREET_NAME', 'LONGITUDE', 'LATITUDE']:
        if col in chunk.columns:
            chunk[col] = chunk[col].fillna('')

    # ---------- 修正类型 ----------
    if 'POSTCODE' in chunk.columns:
        chunk['POSTCODE'] = chunk['POSTCODE'].astype(str)

    # 将经纬度转换为 float，避免 dtype 警告
    for col in ['LONGITUDE', 'LATITUDE']:
        if col in chunk.columns:
            chunk[col] = pd.to_numeric(chunk[col], errors='coerce')

    # 街道名大写
    chunk['STREET_NAME_U'] = chunk['STREET_NAME'].str.strip().str.upper()

    # ---------- 先按 state / suburb / postcode 缩小范围 ----------
    chunk = chunk.merge(
        df_address_filtered[df_match_cols].drop_duplicates(),
        left_on=gnaf_match_cols,
        right_on=df_match_cols,
        how='inner'
    )

    if chunk.empty:
        continue

    # ---------- 只处理未匹配的地址 ----------
    df_unmatched = df_address_filtered[df_address_filtered['LONGITUDE'].isna()]
    if df_unmatched.empty:
        break

    # ---------- 按 state/suburb/postcode 分组向量化匹配 ----------
    grouped = chunk.groupby(['STATE', 'LOCALITY_NAME', 'POSTCODE'])

    for (state, suburb, postcode), gnaf_group in grouped:

        # 匹配对应区域的未匹配地址
        mask = (
            (df_unmatched['state'] == state) &
            (df_unmatched['suburb'] == suburb) &
            (df_unmatched['postcode'] == postcode)
        )
        df_area = df_unmatched[mask]

        if df_area.empty:
            continue

        # 遍历当前区域的 GNAF 地址（少量，向量化匹配）
        for _, g in gnaf_group.iterrows():
            st = g['STREET_NAME_U']
            if not st or len(st) < 3:
                continue

            # ---------- 仅用街道名匹配 ----------
            street_regex = re.compile(rf'\b{re.escape(st)}\b')
            final_mask = df_area['address_full_u'].str.contains(street_regex, na=False)

            if final_mask.any():
                # 找到需要更新的索引
                indices = df_area.index[final_mask]

                # 赋值经纬度，保证 float 类型
                df_address_filtered.loc[indices, 'LONGITUDE'] = g['LONGITUDE']
                df_address_filtered.loc[indices, 'LATITUDE'] = g['LATITUDE']

# =============================
# 6️⃣ 匹配结果统计
# =============================
matched_count = df_address_filtered['LONGITUDE'].notna().sum()
print(f"匹配到经纬度的地址数量: {matched_count} / {len(df_address_filtered)}")



匹配到经纬度的地址数量: 60643 / 70383


In [ ]:
# 三位数的postcode前面补0
df_address_filtered['postcode'] = (
    df_address_filtered['postcode']
    .astype(str)
    .str.strip()
    .apply(lambda x: '0' + x if len(x) == 3 else x)
)

In [14]:
# =============================
# 8️⃣ 第二轮兜底匹配（仅 postcode + street name）
# =============================

# 仅取第一轮未匹配成功的地址
df_unmatched_2 = df_address_filtered[
    df_address_filtered['LONGITUDE'].isna()
].copy()

print(f"第二轮待匹配地址数: {len(df_unmatched_2)}")

if not df_unmatched_2.empty:

    target_postcodes = df_unmatched_2['postcode'].unique()

    for chunk in pd.read_csv(gnaf_file, sep='|', chunksize=chunksize, dtype=str):

        # ---------- 基础清洗 ----------
        for col in ['POSTCODE', 'STREET_NAME', 'LONGITUDE', 'LATITUDE']:
            if col in chunk.columns:
                chunk[col] = chunk[col].fillna('')

        # ---------- 类型修正 ----------
        chunk['POSTCODE'] = chunk['POSTCODE'].astype(str)

        for col in ['LONGITUDE', 'LATITUDE']:
            chunk[col] = pd.to_numeric(chunk[col], errors='coerce')

        # ---------- 街道名大写 ----------
        chunk['STREET_NAME_U'] = chunk['STREET_NAME'].str.strip().str.upper()

        # ---------- 只保留相关 postcode ----------
        chunk = chunk[chunk['POSTCODE'].isin(target_postcodes)]
        if chunk.empty:
            continue

        # ---------- 按 postcode 分组 ----------
        grouped = chunk.groupby('POSTCODE')

        for postcode, gnaf_group in grouped:

            df_area = df_unmatched_2[df_unmatched_2['postcode'] == postcode]
            if df_area.empty:
                continue

            for _, g in gnaf_group.iterrows():
                st = g['STREET_NAME_U']
                if not st or len(st) < 3:
                    continue

                street_regex = re.compile(rf'\b{re.escape(st)}\b')
                mask = df_area['address_full_u'].str.contains(street_regex, na=False)

                if mask.any():
                    indices = df_area.index[mask]

                    # 👉 原地回写到 df_address_filtered
                    df_address_filtered.loc[indices, 'LONGITUDE'] = g['LONGITUDE']
                    df_address_filtered.loc[indices, 'LATITUDE'] = g['LATITUDE']

        # 🔴 新增：动态缩小工作集（逻辑不变）
        df_unmatched_2 = df_address_filtered[
            df_address_filtered['LONGITUDE'].isna()
        ]

        # 🔴 新增：提前结束
        if df_unmatched_2.empty:
            break
# =============================
# 9️⃣ 第二轮后未匹配地址统计
# =============================

remaining_unmatched = df_address_filtered['LONGITUDE'].isna().sum()
total_count = len(df_address_filtered)

print(f"第二轮后仍未匹配到经纬度的地址数量: {remaining_unmatched} / {total_count}")

df_address_filtered.drop(columns=['address_full_u'], inplace=True)


第二轮待匹配地址数: 9740
第二轮后仍未匹配到经纬度的地址数量: 2522 / 70383


In [15]:

df_sql = (
    df_address_filtered[
        [
            "person_id",
            "membership_id",
            "state",
            "suburb",
            "postcode",
            "address_full",
            "LONGITUDE",
            "LATITUDE",
        ]
    ]
    .rename(
        columns={
            "address_full": "address_line",
            "LONGITUDE": "long",
            "LATITUDE": "lat",
        }
    )
)


In [16]:
df_sql

,person_id,membership_id,state,suburb,postcode,address_line,long,lat
1,49.0,57.0,NSW,FULLERTON COVE,2318,4 Brushtail Place,151.798870,-32.856461
4,142.0,154.0,NSW,LITHGOW,2790,3 Suvla Street,150.135887,-33.493590
8,228.0,276.0,NSW,PORTLAND,2847,46 Wallerawang Road,149.981067,-33.355017
9,235.0,282.0,NSW,BEN BULLEN,2790,Ben-Ay Quarry Road,150.013817,-33.220338
13,309.0,386.0,NSW,LITHGOW,2790,45 Andrew Street,150.124535,-33.477322
...,...,...,...,...,...,...,...,...
172312,349642.0,200914.0,VIC,WENDOUREE,3355,43 Harold Street,143.832324,-37.530033
172313,349648.0,200917.0,QLD,NAMBOUR,4560,106 Panorama Dr,152.966569,-26.635581
172314,349688.0,200930.0,QLD,SPRINGFIELD,4300,10 Montcliff Court,152.913697,-27.657400
172315,349707.0,200937.0,NSW,COOLAMON,2701,84 Wallace Street South,147.193811,-34.813207


In [17]:
import geopandas as gpd
from shapely.geometry import Point

In [18]:
# ==========================================
# 1️⃣ 设置 zip 文件路径和年份
# ==========================================
input_folder = r"\\prdeqs01\QlikData\GEO\input"
year = "21"        # 列名后缀，例如 SA1_CODE21
full_year = "2021" # zip 文件名中的年份，例如 SA1_2021_AUST_SHP_GDA2020.zip
zip_filename = f"SA1_{full_year}_AUST_SHP_GDA2020.zip"
zip_path = os.path.join(input_folder, zip_filename)

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"Zip file not found: {zip_path}")

# ==========================================
# 2️⃣ 读取 ABS shapefile
# ==========================================
gdf_area = gpd.read_file(f"zip://{zip_path}")
print("Shapefile columns:", gdf_area.columns)

# ==========================================
# 3️⃣ 动态找到需要的列
# ==========================================
# 我们需要 SA1~SA4 的 CODE 和 NAME 列
sa_code_cols = [c for c in gdf_area.columns if c.startswith("SA") and c.endswith(year) and "CODE" in c]
sa_name_cols = [c for c in gdf_area.columns if c.startswith("SA") and c.endswith(year) and "NAME" in c]

# geometry 一定要保留
required_cols = sa_code_cols + sa_name_cols + ["geometry"]
print("Columns to extract from shapefile:", required_cols)

# ==========================================
# 4️⃣ 创建 points GeoDataFrame，只对有 long/lat 的行
# ==========================================
df_points = df_sql.dropna(subset=["long", "lat"]).copy()
gdf_points = gpd.GeoDataFrame(
    df_points,
    geometry=gpd.points_from_xy(df_points.long, df_points.lat),
    crs="EPSG:4326"
)

# ==========================================
# 5️⃣ 坐标系对齐
# ==========================================
gdf_points = gdf_points.to_crs(gdf_area.crs)

# ==========================================
# 6️⃣ 空间匹配
# ==========================================
gdf_joined = gpd.sjoin(
    gdf_points,
    gdf_area[required_cols],
    how="left",
    predicate="within"
)

# ==========================================
# 6️⃣a 去除重复 person_id，保证 merge 后行数不增加
# 如果一个点落在多个 polygon 中，只保留第一个匹配
# ==========================================
gdf_joined = gdf_joined.drop_duplicates(subset="person_id")

# ==========================================
# 7️⃣ 合并回原始 df_sql
# ==========================================
df_sql_with_sa = df_sql.merge(
    gdf_joined[["person_id"] + sa_code_cols + sa_name_cols],
    on="person_id",
    how="left"
)

# ==========================================
# 8️⃣ 重命名列为动态统一格式
#    例如：SA1_CODE21 -> SA1_CODE
# ==========================================
rename_dict = {c: c[:-2] for c in sa_code_cols + sa_name_cols}  # 去掉年份后缀
df_sql_with_sa = df_sql_with_sa.rename(columns=rename_dict)

# ==========================================
# 9️⃣ （可选）去掉 geometry 列
# ==========================================
df_sql_with_sa = df_sql_with_sa.drop(columns=["geometry"], errors="ignore")

print("最终 df_sql_with_sa 行数:", len(df_sql_with_sa))


Shapefile columns: Index(['SA1_CODE21', 'CHG_FLAG21', 'CHG_LBL21', 'SA2_CODE21', 'SA2_NAME21',
       'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21',
       'GCC_NAME21', 'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21',
       'AREASQKM21', 'LOCI_URI21', 'geometry'],
      dtype='object')
Columns to extract from shapefile: ['SA1_CODE21', 'SA2_CODE21', 'SA3_CODE21', 'SA4_CODE21', 'SA2_NAME21', 'SA3_NAME21', 'SA4_NAME21', 'geometry']
最终 df_sql_with_sa 行数: 70383


In [19]:
df_sql_with_sa

,person_id,membership_id,state,suburb,postcode,address_line,long,lat,SA1_CODE,SA2_CODE,SA3_CODE,SA4_CODE,SA2_NAME,SA3_NAME,SA4_NAME
0,49.0,57.0,NSW,FULLERTON COVE,2318,4 Brushtail Place,151.798870,-32.856461,11103123221,111031232,11103,111,Stockton - Fullerton Cove,Newcastle,Newcastle and Lake Macquarie
1,142.0,154.0,NSW,LITHGOW,2790,3 Suvla Street,150.135887,-33.493590,10303107026,103031070,10303,103,Lithgow,Lithgow - Mudgee,Central West
2,228.0,276.0,NSW,PORTLAND,2847,46 Wallerawang Road,149.981067,-33.355017,10303107122,103031071,10303,103,Lithgow Surrounds,Lithgow - Mudgee,Central West
3,235.0,282.0,NSW,BEN BULLEN,2790,Ben-Ay Quarry Road,150.013817,-33.220338,10303107128,103031071,10303,103,Lithgow Surrounds,Lithgow - Mudgee,Central West
4,309.0,386.0,NSW,LITHGOW,2790,45 Andrew Street,150.124535,-33.477322,10303107004,103031070,10303,103,Lithgow,Lithgow - Mudgee,Central West
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70378,349642.0,200914.0,VIC,WENDOUREE,3355,43 Harold Street,143.832324,-37.530033,20101100823,201011008,20101,201,Wendouree - Miners Rest,Ballarat,Ballarat
70379,349648.0,200917.0,QLD,NAMBOUR,4560,106 Panorama Dr,152.966569,-26.635581,31607154838,316071548,31607,316,Nambour,Nambour,Sunshine Coast
70380,349688.0,200930.0,QLD,SPRINGFIELD,4300,10 Montcliff Court,152.913697,-27.657400,31004130319,310041303,31004,310,Springfield,Springfield - Redbank,Ipswich
70381,349707.0,200937.0,NSW,COOLAMON,2701,84 Wallace Street South,147.193811,-34.813207,11303127129,113031271,11303,113,Wagga Wagga Surrounds,Wagga Wagga,Riverina


In [20]:
# 获取当前日期，格式为 ddmmyyyy
today_str = datetime.now().strftime("%d%m%Y")

# 构建文件名
file_name = f"df_sql_with_sa_{today_str}.csv"

# 构建完整路径
file_path = os.path.join(output_folder, file_name)

# 输出 CSV
df_sql_with_sa.to_csv(file_path, index=False, encoding='utf-8-sig')  # encoding='utf-8-sig' 可避免中文乱码

print(f"CSV 文件已保存到: {file_path}")


CSV 文件已保存到: \\prdeqs01\QlikData\GEO\output\df_sql_with_sa_09012026.csv
